In [1]:
# !pip install absl-py langdetect nltk immutabledict datasets bitsandbytes

In [2]:
import torch
torch.__version__

'2.4.1+cu121'

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from tqdm import tqdm
import json
from peft import prepare_model_for_kbit_training, PeftConfig, PeftModel

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the dataset
dataset = load_dataset("google/IFEval")

model_name = 'mistralai/Mistral-7B-v0.3'
checkpoint_path = '../models/model10/output_lima/checkpoint-100'
model_name = "Qwen/Qwen2.5-7B"
checkpoint_path = "../models/model11/output_oasst1/checkpoint-2000"

# Step 1: Load the tokenizer and model with quantization
# model_name = "models/model1"  # Near 3B model (smallest available Qwen model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_fast=True,
    padding_side='left'
)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Enable loading the model in 4-bit precision
    bnb_4bit_quant_type="nf4",            # Specify quantization type as Normal Float 4
    bnb_4bit_compute_dtype=getattr(torch, "bfloat16"), # Set computation data type
    bnb_4bit_use_double_quant=True,       # Use double quantization for better accuracy
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

peft_config = PeftConfig.from_pretrained(checkpoint_path)
model = PeftModel.from_pretrained(model, checkpoint_path)
model.config.pad_token_id = tokenizer.eos_token_id

# Disable gradients to save memory and computation
model.eval()
torch.set_grad_enabled(False)  # Disable gradient computation globally

# Prepare the output file
output_dir = "ifeval_responses"
os.makedirs(output_dir, exist_ok=True)
num_model = checkpoint_path.split('/')[-3].split("model")[-1]
output_file = os.path.join(output_dir, f"input_response_data{num_model}.jsonl")

# Batch processing
batch_size = 8  # Adjust based on your GPU memory capacity
max_length = 128  # Limit output length to avoid excessive memory usage

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# import os
# import torch
# from datasets import load_dataset
# from transformers import (
#     AutoModelForCausalLM,
#     AutoTokenizer,
#     BitsAndBytesConfig
# )
# from tqdm import tqdm
# import json

# # Device
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Load the dataset
# dataset = load_dataset("google/IFEval")

# # Model name
# model_name = "../models/model10/output_lima/checkpoint-1000"
# # model_name = "Qwen/Qwen2.5-7B"

# # Load the tokenizer for Mistral
# tokenizer = AutoTokenizer.from_pretrained(
#     model_name,
#     add_eos_token=True,
#     use_fast=True,
#     padding_side='left',
# )
# tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS token

# # Quantization configuration using bitsandbytes library
# compute_dtype = getattr(torch, "bfloat16")
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=True,
# )

# # Load the pre-trained model with the specified quantization configuration
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     device_map="auto"
# )
# model.config.pad_token_id = tokenizer.pad_token_id  # Set the model's padding token ID

# # Disable gradients to save memory and computation
# model.eval()
# torch.set_grad_enabled(False)  # Disable gradient computation globally

# # Prepare the output file
# output_dir = "data"
# os.makedirs(output_dir, exist_ok=True)
# output_file = os.path.join(output_dir, "input_response_data8.jsonl")

# # Batch processing
# batch_size = 8  # Adjust based on your GPU memory capacity
# max_length = 128  # Limit output length to avoid excessive memory usage

In [5]:
def tokenize_batch_prompt(batch_prompts):
    # We add at the beginning of each prompt the token for the end of the previous prompt "<Prompt>:"
    # and at the end " \n<Response>:"
    batch_prompts = [f"<Prompt>: {prompt} \n<Response>:" for prompt in batch_prompts]
    return tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to(device)

In [6]:
with open(output_file, 'w') as f:
    # Process in batches
    for i in tqdm(range(0, len(dataset['train']), batch_size), desc="Processing Batches", unit="batch"):
        try:
            if (i + batch_size) > len(dataset['train']):
                batch_prompts = dataset['train']['prompt'][i:len(dataset['train'])]
            else:
                batch_prompts = dataset['train']['prompt'][i:i + batch_size]
            # Tokenize inputs
            # inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to(device)
            inputs = tokenize_batch_prompt(batch_prompts)
            
            # Generate responses
            with torch.no_grad():  # Ensure gradients are disabled during generation
                if max_length:
                    outputs = model.generate(**inputs, max_new_tokens=max_length)
                else:
                    outputs = model.generate(**inputs)
            # Decode responses and remove the prompt
            responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
            for j in range(len(responses)):
                responses[j] = responses[j][len(f"<Prompt>: {batch_prompts[j]} \n<Response>:"):]
            # Write each response directly to the file
            for prompt, response in zip(batch_prompts, responses):
                f.write(json.dumps({"prompt": prompt, "response": response}) + '\n')
        except Exception as e:
            print(f"Error processing batch {i}: {e}")

print(f"Responses saved to {output_file}")

Processing Batches:   0%|          | 0/68 [00:00<?, ?batch/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
2024-11-24 14:21:19.442333: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 14:21:19.451644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 14:21:19.460080: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 14:21

Responses saved to data/input_response_data11.jsonl
